In [65]:
with open('./day16_input.txt') as f:
    compressed = f.read()

In [82]:
compresed_to_binary = {
    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3': '0011',
    '4': '0100',
    '5': '0101',
    '6': '0110',
    '7': '0111',
    '8': '1000',
    '9': '1001',
    'A': '1010',
    'B': '1011',
    'C': '1100',
    'D': '1101',
    'E': '1110',
    'F': '1111',
}

In [83]:
def uncompress(compressed: str):
    return ''.join([compresed_to_binary[c] for c in compressed if c != '\n'])

In [84]:
binary = uncompress(compressed)
binary

'101000100000110101111001000000000100001011110001001001110100000000010001100101010101010010010001100000001000101110000000001011110001110001100000101100100000000000110000001100100111101011110010110011000010010010001010101010000000000011100111110011011101011100100110111100111101011110001111010010010110011011110101011100011010001100000000101110100101010011010110011010001110001001010001100100100100100100100110010100010110000010000000001111101010100111011110010101100010101000011000000000010010000001001010110011100101001111001001010101001010110011100101001111001001010011000110010110011011110111110011000110001111110100010011011001101110111101000100110110010110111010110001000100000000010111111011001110010001010101001110000000000110100010100111110000111010011010110011011110010110010001101100100000000011010010001010000000000101010111100110011001000010101100110011001000010000100110111000110101101111000011110001001010011000000001000101001011001001110100110010001010110010100000000001001011101100011

In [85]:
def parse_version(bi: str, cur: int):
    return int(bi[cur:cur + 3], 2), cur + 3

def parse_type_id(bi: str, cur: int):
    return int(bi[cur:cur+3], 2), cur + 3

def parse_literal(bi: str, cur: int):
    to_continue = True
    val = ''
    while to_continue:
        to_continue = bi[cur] == '1'
        val += bi[cur+1: cur+5]
        cur += 5
    return int(val, 2), cur

def parse_length_type_id(bi: str, cur: int):
    return int(bi[cur], 2), cur + 1
    
def parse_number_of_packets(bi: str, cur: int):
    return int(bi[cur:cur+11], 2), cur+11

def parse_len_of_packets(bi: str, cur: int):
    return int(bi[cur:cur+15], 2), cur+15

In [86]:
from typing import List

def do_operation(type_id: int, other_vals: List[int]):
    """
    Packets with type ID 0 are sum packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
    Packets with type ID 1 are product packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
    Packets with type ID 2 are minimum packets - their value is the minimum of the values of their sub-packets.
    Packets with type ID 3 are maximum packets - their value is the maximum of the values of their sub-packets.
    Packets with type ID 5 are greater than packets - their value is 1 if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
    Packets with type ID 6 are less than packets - their value is 1 if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
    Packets with type ID 7 are equal to packets - their value is 1 if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
    """
    if type_id == 0:
        return sum(other_vals)
    elif type_id == 1:
        prod = 1
        for v in other_vals:
            prod *= v
        return prod
    elif type_id == 2:
        return min(other_vals)
    elif type_id == 3:
        return max(other_vals)
    elif type_id == 5:
        a_val, b_val = other_vals
        return 1 if a_val > b_val else 0
    elif type_id == 6:
        a_val, b_val = other_vals
        return 1 if a_val < b_val else 0
    elif type_id == 7:
        a_val, b_val = other_vals
        return 1 if a_val == b_val else 0
    raise ValueError(type_id)
    
    

In [87]:
version_sum = 0


def parse_packet(binary: str, cur: int = 0):
    global version_sum
    v, cur = parse_version(binary, cur)
    version_sum += v
    t_id, cur = parse_type_id(binary, cur)
    if t_id == 4:
        lit, cur = parse_literal(binary, cur)
        return lit, cur
    else:
        lt, cur = parse_length_type_id(binary, cur)
        if lt == 0:
            subpacket_len, cur = parse_len_of_packets(binary, cur)
            other_cur = cur
            other_vals = []
            while other_cur - cur < subpacket_len:
                other_val, other_cur = parse_packet(binary, other_cur)
                other_vals.append(other_val)
            return do_operation(t_id, other_vals), other_cur
        else:
            num_packets, cur = parse_number_of_packets(binary, cur)
            other_vals = []
            for i in range(num_packets):
                other_val, cur = parse_packet(binary, cur)
                other_vals.append(other_val)
            return do_operation(t_id, other_vals), cur

In [92]:
version_sum = 0 
answer, _ = parse_packet(binary)
print('PART 1', version_sum)
print('PART 2', answer)

PART 1 854
PART 2 186189840660
